In [3]:
import nest_asyncio
from browser_use import Agent
nest_asyncio.apply()

INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://github.com/gregpr07/browser-use for more information.


In [1]:
from langchain_google_vertexai.model_garden import ChatAnthropicVertex
from langchain_google_vertexai import ChatVertexAI

In [2]:
project = "ai-call-bot-440714"
location = "us-east5"

# Initialise the Model
model = ChatAnthropicVertex(
    model_name="claude-3-5-sonnet-v2@20241022",
    project=project,
    location=location,
)

In [4]:
agent = Agent(
    task="Go to https://wargaming.com/en/careers/ and extract the list of all paths of the vacancies by href, if needed click next page button",
    llm=model,
)
result = await agent.run()

INFO     [agent] 🚀 Starting task: Go to https://wargaming.com/en/careers/ and extract the list of all paths of the vacancies by href, if needed click next page button
INFO     [agent] 
📍 Step 1
INFO     [agent] 🤷 Eval: Unknown - Starting fresh on a blank page
INFO     [agent] 🧠 Memory: Need to go to Wargaming careers page and extract vacancy hrefs
INFO     [agent] 🎯 Next goal: Navigate to the Wargaming careers page
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://wargaming.com/en/careers/"}}
ERROR    [agent] ❌ Result failed 1/3 times:
 Error executing action go_to_url: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://wargaming.com/en/careers/", waiting until "load"

INFO     [agent] 
📍 Step 2
INFO     [agent] ⚠ Eval: Failed - The initial URL navigation timed out, but we landed on the correct page anyway
INFO     [agent] 🧠 Memory: We are on the Wargaming careers page and need to click on Latest job opportunities to find the vacancy listings
INFO    

In [2]:
from playwright.async_api import async_playwright
import asyncio

async def extract_vacancy_paths():
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=[
            '--no-sandbox',
            '--disable-blink-features=AutomationControlled',
            '--disable-infobars',
            '--disable-background-timer-throttling',
            '--disable-popup-blocking',
            '--disable-backgrounding-occluded-windows',
            '--disable-renderer-backgrounding',
            '--disable-window-activation',
            '--disable-focus-on-load',
            '--no-first-run',
            '--no-default-browser-check',
            '--no-startup-window',
            '--window-position=0,0',
            '--window-size=1280,1000',
            '--disable-web-security',
            '--disable-site-isolation-trials',
            '--disable-features=IsolateOrigins,site-per-process'
        ])
        page = await browser.new_page()

        vacancy_paths = []


        try:
            await page.goto("https://wargaming.com/en/careers/", timeout=30000)
            await page.wait_for_load_state('networkidle')
        except Exception as e:
            print(f"Navigation timeout occurred: {e}")
            # Continue execution as the page might still be usable



        # Wait for and click the jobs list link
        try:
            await page.wait_for_selector('a[href="#jobs_list"]', timeout=30000)
            await page.click('a[href="#jobs_list"]')
            await page.wait_for_load_state('networkidle')
        except Exception as e:
            print(f"Selector 'a[href='#jobs_list']' not found: {e}")
            # Continue execution as the page might still be usable

        # Wait for job listings to appear
        try:
            await page.wait_for_selector('.job-listing', timeout=30000)
        except Exception as e:
            print(f"Selector '.job-listing' not found: {e}")
            # Continue execution as the page might still be usable

        # Try different selectors for job links
        selectors = [
            '.job-listing a',
            'a[href*="/careers/vacancy/"]',
            '[data-vacancy-link]',
        ]

        for selector in selectors:
            try:
                links = await page.eval_on_selector_all(
                    selector,
                    "elements => elements.map(el => el.getAttribute('href'))"
                )
                if links:
                    vacancy_paths.extend(links)
                    break
            except Exception as e:
                print(f"Selector {selector} failed: {e}")
                continue
        # Filter out None values and duplicates
        return list(set([path for path in vacancy_paths if path]))

In [16]:
paths = await extract_vacancy_paths()
print(f"Found {len(paths)} unique vacancy paths:")
for path in paths:
    print(path)

Navigation timeout occurred: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://wargaming.com/en/careers/", waiting until "load"

Selector 'a[href='#jobs_list']' not found: Timeout 30000ms exceeded.
=========================== logs ===========================
"load" event fired
Selector '.job-listing' not found: Page.wait_for_selector: Timeout 30000ms exceeded.
Call log:
  - waiting for locator(".job-listing") to be visible

Found 0 unique vacancy paths:


In [3]:
async def extract_vacancy_details(url: str) -> str:
    """
    Extract HTML content from a specific vacancy page.

    Args:
        url (str): Full URL of the vacancy page

    Returns:
        str: HTML content of the page
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=[
                '--no-sandbox',
                '--disable-blink-features=AutomationControlled',
                '--disable-infobars',
                # ... keeping same browser arguments as before ...
            ],
        )
        page = await browser.new_page()

        try:
            try:
                await page.goto(url, timeout=30000)
                await page.wait_for_load_state('networkidle')
            except Exception as e:
                print(e)
            # Get the full HTML content
            html_content = await page.content()
            return html_content

        except Exception as e:
            print(f"Error extracting vacancy details from {url}: {e}")
            return ""
        finally:
            await browser.close()

# Example usage:
url = "https://wargaming.com/en/careers/vacancy_3022040_warsaw/"
html_content = await extract_vacancy_details(url)
print(f"Retrieved {len(html_content)} characters of HTML content")

Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://wargaming.com/en/careers/vacancy_3022040_warsaw/", waiting until "load"

Retrieved 324487 characters of HTML content


In [18]:
from pydantic import BaseModel, Field


class Vacancy(BaseModel):
    """Information about vacancy."""

    title: str = Field(description="Title vacancy from html")
    description: str = Field(description="description vacancy from html")
    skills: list[str] | None = Field(default=None, description="Skills vacancy from html")
    required_experience: str | None = Field(default=None, description="required experience vacancy from html")
    location: str = Field(description="location of vacancy from html")

    level: str | None = Field(default=None, description="required level vacancy from html")
    salary_min: int | None = Field(default=None, description="min salary vacancy from html")
    salary_max: int | None = Field(default=None, description="max salary vacancy from html")
    salary_currency: str | None = Field(default=None, description="currency salary vacancy from html")

    # Дополнительная информация
    responsibilities: str | None = Field(default=None, description="responsibilities vacancy from html")
    benefits: str | None = Field(default=None, description="benefits vacancy from html")
    remote_type: str | None = Field(default=None, description="type of remote vacancy from html: office, hybrid, remote")

    # Информация o компании
    department: str | None = Field(default=None, description="Unit of company vacanct from html")

TypeError: EnumType.__call__() takes from 2 to 3 positional arguments but 4 were given

In [10]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model="gemini-1.5-flash-002",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
)


In [11]:
structured_llm = llm.with_structured_output(Vacancy)

In [12]:
vacancy = structured_llm.invoke(html_content)

In [14]:
from pprint import pprint
pprint(vacancy.dict())

{'description': 'Wargaming is seeking a talented Senior Infrastructure '
                'Engineer with extended knowledge of Python and DevOps to join '
                'our Game Engine Development Team. In this role, you will help '
                'improve and evolve our Continuous Integration and Delivery '
                'processes with a focus on automation and seamless '
                'integration.',
 'location': 'Warsaw, Poland',
 'required_experience': 'Strong experience with Python',
 'skills': ['Python',
            'DevOps',
            'CI/CD',
            'automation',
            'Git',
            'Bitbucket',
            'SVN'],
 'title': 'Senior Infrastructure Engineer (Python) (Game Engine Development '
          'Team)'}
